# Acrobot

## A2C Agent 

In [1]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "Acrobot-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "Acrobot-v1":
    gamma = 0.95
    lr = 7e-4

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -343.0
Episode 4: -500.0
Episode 5: -286.0
Episode 6: -500.0
Episode 7: -378.0
Episode 8: -379.0
Episode 9: -500.0
Episode 10: -500.0
Episode 11: -500.0
Episode 12: -429.0
Episode 13: -500.0
Episode 14: -500.0
Episode 15: -500.0
Episode 16: -500.0
Episode 17: -500.0
Episode 18: -489.0
Episode 19: -500.0
Episode 20: -500.0
Episode 21: -496.0
Episode 22: -299.0
Episode 23: -331.0
Episode 24: -373.0
Episode 25: -340.0
Episode 26: -325.0
Episode 27: -281.0
Episode 28: -340.0
Episode 29: -445.0
Episode 30: -381.0
Episode 31: -191.0
Episode 32: -274.0
Episode 33: -308.0
Episode 34: -258.0
Episode 35: -500.0
Episode 36: -262.0
Episode 37: -231.0
Episode 38: -316.0
Episode 39: -237.0
Episode 40: -191.0
Episode 41: -405.0
Episode 42: -276.0
Episode 43: -281.0
Episode 44: -325.0
Episode 45: -219.0
Episode 46: -500.0
Episode 47: -129.0
Episode 48: -208.0
Episode 49: -225.0
Episode 50: -196.0
Episode 51: -135.0
Episode 52: -350.0
Epi

In [2]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (KL) 

In [1]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = "Acrobot-v1"
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent
if kl_env == "Acrobot-v1":
    gamma = 0.95
    lr = 5e-3
    beta = 0.8

agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration
    beta += np.random.random()*0.1
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward  <= -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2
        
    if episode % 10 == 0:
        state_adv[0] += (np.random.random()-0.5)*0.5
        state_adv[1] += (np.random.random()-0.5)*0.5
        state_adv[2] += (np.random.random()-0.5)*0.5

    policy_loss = agent.compute_policy_loss_kl(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -453.3333333333333
Episode 3: -426.0
Episode 4: -474.3333333333333
Episode 5: -410.0
Episode 6: -366.6666666666667
Episode 7: -275.3333333333333
Episode 8: -359.0
Episode 9: -292.0
Episode 10: -352.3333333333333
Episode 11: -331.0
Episode 12: -231.66666666666666
Episode 13: -310.0
Episode 14: -288.3333333333333
Episode 15: -287.3333333333333
Episode 16: -279.0
Episode 17: -246.33333333333334
Episode 18: -243.0
Episode 19: -259.3333333333333
Episode 20: -222.33333333333334
Episode 21: -192.0
Episode 22: -195.0
Episode 23: -174.0
Episode 24: -242.0
Episode 25: -178.66666666666666
Episode 26: -261.3333333333333
Episode 27: -136.33333333333334
Episode 28: -184.0
Episode 29: -152.66666666666666
Episode 30: -146.0
Episode 31: -131.33333333333334
Episode 32: -178.66666666666666
Episode 33: -147.0
Episode 34: -188.33333333333334
Episode 35: -141.0
Episode 36: -145.33333333333334
Episode 37: -145.33333333333334
Episode 38: -139.0
Episode 39: -115.3

Episode 295: -90.66666666666667
Episode 296: -104.33333333333333
Episode 297: -155.0
Episode 298: -91.0
Episode 299: -93.0


In [2]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Wasserstein)

In [3]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "Acrobot-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "Acrobot-v1":
    gamma = 0.95
    lr = 5e-3
    
agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 300
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
    
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration 
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward < -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2
        
    total_adv_diff += max(abs(state_adv[1] - state_adv[0]), abs(state_adv[2] - state_adv[0]), abs(state_adv[2] - state_adv[1]))
    beta = total_adv_diff/episode
    beta += 0.1
    policy_loss = agent.compute_policy_loss_wass(first_state, state_adv, beta)

    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -466.0
Episode 4: -500.0
Episode 5: -476.3333333333333
Episode 6: -500.0
Episode 7: -500.0
Episode 8: -444.6666666666667
Episode 9: -420.6666666666667
Episode 10: -365.6666666666667
Episode 11: -341.6666666666667
Episode 12: -284.0
Episode 13: -257.3333333333333
Episode 14: -302.0
Episode 15: -223.33333333333334
Episode 16: -219.0
Episode 17: -179.0
Episode 18: -211.66666666666666
Episode 19: -170.0
Episode 20: -181.66666666666666
Episode 21: -205.33333333333334
Episode 22: -165.66666666666666
Episode 23: -180.66666666666666
Episode 24: -128.0
Episode 25: -256.3333333333333
Episode 26: -204.0
Episode 27: -211.33333333333334
Episode 28: -148.33333333333334
Episode 29: -227.66666666666666
Episode 30: -178.0
Episode 31: -185.0
Episode 32: -172.33333333333334
Episode 33: -189.66666666666666
Episode 34: -159.33333333333334
Episode 35: -167.33333333333334
Episode 36: -143.33333333333334
Episode 37: -171.0
Episode 38: -310.0
Epi

Episode 285: -152.0
Episode 286: -291.0
Episode 287: -173.33333333333334
Episode 288: -209.0
Episode 289: -151.0
Episode 290: -135.66666666666666
Episode 291: -136.0
Episode 292: -136.66666666666666
Episode 293: -177.33333333333334
Episode 294: -184.33333333333334
Episode 295: -157.33333333333334
Episode 296: -152.66666666666666
Episode 297: -148.33333333333334
Episode 298: -183.33333333333334
Episode 299: -148.33333333333334


In [4]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')